In [1]:
import pandas as pd
from pathlib import Path

csv_path = Path(r"C:\Users\pc\Root\programming\whatsapp_sender\samples\recipients.csv")
if not csv_path.exists():
    raise FileNotFoundError(f"CSV not found: {csv_path}")

xlsx_path = csv_path.with_suffix(".xlsx")

# Read CSV (preserve as strings to avoid type surprises) and write Excel
df = pd.read_csv(csv_path, dtype=str)
df.to_excel(xlsx_path, index=False, engine="openpyxl")

print(f"Converted {csv_path} -> {xlsx_path}")

Converted C:\Users\pc\Root\programming\whatsapp_sender\samples\recipients.csv -> C:\Users\pc\Root\programming\whatsapp_sender\samples\recipients.xlsx


In [2]:
payload = {
  "messaging_product": "whatsapp",
  "to": "201113025205",
  "type": "template",
  "template": {
    "name": "campaign",
    "language": {
      "code": "ar"
    },
    "components": [
      {
        "type": "header",
        "parameters": [
          {
            "type": "image",
            "image": {
              "id": "2095534587856495"
            }
          }
        ]
      },
      {
        "type": "button",
        "sub_type": "copy_code",
        "index": "0",
        "parameters": [
          { "type": "coupon_code", "coupon_code": "RC20" }
        ]
      }
    ]
  }
}


In [3]:
phones = ["201113025205", "201018999514"]

In [8]:
%pip install dotenv

  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import copy
from typing import Iterable, List, Dict, Any

def duplicate_payloads(payload: Dict[str, Any], recipients: Iterable) -> List[Dict[str, Any]]:
    """
    Duplicate a payload for each recipient in `recipients`, setting the 'to' field.

    Args:
        payload: original payload dict
        recipients: iterable of phone numbers (converted to str)

    Returns:
        List of deep-copied payload dicts with updated 'to' values.
    """
    out: List[Dict[str, Any]] = []
    for r in recipients:
        p = copy.deepcopy(payload)
        p["to"] = str(r)
        out.append(p)
    return out

# Example usage:
new_payloads = duplicate_payloads(payload, phones)

In [9]:
from __future__ import annotations

import argparse
import csv
import json
import os
import sys
import time
from pathlib import Path
from typing import List, Optional

from dotenv import load_dotenv

from whatsapp_client import MediaCache, WhatsAppClient, WhatsAppConfig


def load_config() -> WhatsAppConfig:
    load_dotenv(override=False)
    token = os.getenv("WHATSAPP_TOKEN")
    phone_id = os.getenv("WHATSAPP_PHONE_NUMBER_ID")
    api_version = os.getenv("WHATSAPP_API_VERSION", "v20.0")
    if not token or not phone_id:
        print("Missing token or phone number ID. Set in .env or pass flags.", file=sys.stderr)
        sys.exit(2)
    return WhatsAppConfig(token=token, phone_number_id=phone_id, api_version=api_version)


config = load_config()



In [10]:
from whatsapp_client import MediaCache, WhatsAppClient, WhatsAppConfig
client = WhatsAppClient(config=config, media_cache=MediaCache(Path("media_cache.json")))

import concurrent.futures

def send_payload(p):
    return client.send_message(p)

with concurrent.futures.ThreadPoolExecutor() as executor:
    responses = list(executor.map(send_payload, new_payloads))